# Librerías

In [8]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier    # Clasificador de Bosques aleatorios
from sklearn.model_selection import GridSearchCV       # Rejilla de hiperparametros

# Lectura base de datos

In [60]:
Ocurrencia_Incendios = pd.read_csv("Base_datos1.csv")
Ocurrencia_Incendios.rename(columns={"Temperatur": "Temperatura"}, inplace=True) # Indica si la modificación debe realizarse directamente en el DataFrame original o si debe devolver un nuevo DataFrame con los cambios aplicados
Ocurrencia_Incendios = Ocurrencia_Incendios.drop(columns = ["FECHA"])
Ocurrencia_Incendios

,LATITUD,LONGITUD,Temperatura,Pmm_radar,DEM_AMVA,Ocurrencia
0,6.052000,-75.619000,23.2313,0.000000,1897,1
1,6.167930,-75.643800,25.5000,0.000000,1798,0
2,6.186000,-75.641000,25.1730,0.000000,1719,1
3,6.167930,-75.643800,25.3000,0.000000,1798,0
4,6.079000,-75.634000,23.8933,0.000000,1816,1
...,...,...,...,...,...,...
5803,6.238000,-75.627000,26.4479,18.074453,1769,1
5804,6.255245,-75.542478,26.9500,10.092163,1877,0
5805,6.254000,-75.536000,24.4348,9.733376,2109,1
5806,6.271891,-75.631123,25.0000,14.298922,1895,0


In [61]:
'''Recordar que es importante que separamos el DataFrame en muestras aleatorias,
Acá un ejemplo de como se pueden separar: '''

from random import sample

print(Ocurrencia_Incendios.columns[:-1], "\n")

columnas_muestreo = list(set(Ocurrencia_Incendios.columns[:-1]))
print(sample(columnas_muestreo, 3))    # No se pone la ocurrencia

Index(['LATITUD', 'LONGITUD', 'Temperatura', 'Pmm_radar', 'DEM_AMVA'], dtype='object') 

['LONGITUD', 'Temperatura', 'LATITUD']


# Exploración hiperparametros para el modelos de arboles aleatorios

In [62]:
'''¿Cuales son los hiperparametros de este modelo?,
A continuación tengo un diccionario de todos los hiperparametros que pueden ser cambiados para ajustar mi modelo: '''

bosque = RandomForestClassifier()
bosque.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [68]:
'''Le puedo agregar más hiperparametros'''

parametros = {"criterion": ("gini", "entropy"),
              "n_estimators" : (10, 20, 30),              # Generar 10, 20, 30 arboles de deciones
              "max_samples" : (1/3, 2/3),                 # Porcentaje de datos para crear los muestreos
              "max_features" : (None, "sqrt", "log2")}    # None = sin restricción, sqrt = Raiz cuadrada y log2 = 


rejilla = GridSearchCV(bosque, 
                       parametros, 
                       scoring="accuracy",
                       cv=5)

rejilla.fit(Ocurrencia_Incendios[Ocurrencia_Incendios.columns[:-1]].values, 
            Ocurrencia_Incendios["Ocurrencia"].values)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_features': (None, 'sqrt', 'log2'),
                         'max_samples': (0.3333333333333333,
                                         0.6666666666666666),
                         'n_estimators': (10, 20, 30)},
             scoring='accuracy')

In [72]:
rejilla.cv_results_["params"]

[{'criterion': 'gini',
  'max_features': None,
  'max_samples': 0.3333333333333333,
  'n_estimators': 10},
 {'criterion': 'gini',
  'max_features': None,
  'max_samples': 0.3333333333333333,
  'n_estimators': 20},
 {'criterion': 'gini',
  'max_features': None,
  'max_samples': 0.3333333333333333,
  'n_estimators': 30},
 {'criterion': 'gini',
  'max_features': None,
  'max_samples': 0.6666666666666666,
  'n_estimators': 10},
 {'criterion': 'gini',
  'max_features': None,
  'max_samples': 0.6666666666666666,
  'n_estimators': 20},
 {'criterion': 'gini',
  'max_features': None,
  'max_samples': 0.6666666666666666,
  'n_estimators': 30},
 {'criterion': 'gini',
  'max_features': 'sqrt',
  'max_samples': 0.3333333333333333,
  'n_estimators': 10},
 {'criterion': 'gini',
  'max_features': 'sqrt',
  'max_samples': 0.3333333333333333,
  'n_estimators': 20},
 {'criterion': 'gini',
  'max_features': 'sqrt',
  'max_samples': 0.3333333333333333,
  'n_estimators': 30},
 {'criterion': 'gini',
  'max_f

In [71]:
print("Con las siguientes variables, la ocurrencia será: ", mejor_bosque.predict([[6.008000, -75.507000, 21.4479, 0, 1769]]))

Con las siguientes variables, la ocurrencia será:  [1]


In [69]:
print("El valor mayor fue el mejor modelo :", rejilla.cv_results_["rank_test_score"])
print("\nresultado de la validación cruzada para los 12 modelos:\n", rejilla.cv_results_["mean_test_score"])
print("\nEl mejor Score fue: ",rejilla.best_score_)
print("\nLos mejores estimadores son: ", rejilla.best_params_)

El valor mayor fue el mejor modelo : [36 34 33 30 26 19 29 24 22  5 11  6 35 15 17 10  9  7 32 31 28 14 20 13
 16 21 18 25  4  2 27 23 12  8  1  3]

resultado de la validación cruzada para los 12 modelos:
 [0.95504958 0.9598693  0.96124743 0.96314145 0.9658977  0.96744735
 0.96520953 0.96624134 0.96658676 0.97244126 0.96968457 0.97209599
 0.95935473 0.96830927 0.96796444 0.97054649 0.97089028 0.9712357
 0.96124876 0.96210949 0.9653815  0.96882606 0.9674466  0.96916955
 0.96796548 0.9669307  0.96796399 0.96606878 0.9724417  0.97312869
 0.9657284  0.96658602 0.96934048 0.97106477 0.97433484 0.97295806]

El mejor Score fue:  0.9743348439902096

Los mejores estimadores son:  {'criterion': 'entropy', 'max_features': 'log2', 'max_samples': 0.6666666666666666, 'n_estimators': 20}


Buscamos optimizar el accuracy

In [70]:
# rejilla.predict es igual a rejilla.best_estimator_

print("Con las siguientes variables, la ocurrencia será: ", rejilla.predict([[6.1846438, -75.6209755, 30, 0, 1769]]))

mejor_bosque = rejilla.best_estimator_
print("Con las siguientes variables, la ocurrencia será: ", mejor_bosque.predict([[6.1846438, -75.6209755, 30, 0, 1769]]))

Con las siguientes variables, la ocurrencia será:  [0]
Con las siguientes variables, la ocurrencia será:  [0]


# Aplicando un solo Bosque aleatorio (Clasificador, para predecirme 0 o 1)

In [38]:
bosque = RandomForestClassifier(n_estimators=30,        # Generar 100 arboles de deciones
                               criterion="entropy",
                               max_features="sqrt",
                               bootstrap=True,
                               max_samples=2/3,         # Porcentaje de datos para crear los muestreos
                               oob_score=True)

bosque.fit(Ocurrencia_Incendios[Ocurrencia_Incendios.columns[:-1]].values, Ocurrencia_Incendios["Ocurrencia"].values)

print(bosque.predict([[6.1846438, -75.6209755, 30, 0, 1769]]))

print("% de items clasificados correctamente", bosque.score(Ocurrencia_Incendios[Ocurrencia_Incendios.columns[:-1]].values, Ocurrencia_Incendios["Ocurrencia"].values))
# Evaluar modelo con los datos que se usaron para crear los arboles osea 2/3 de los datos

print("% de items clasificados correctamente", bosque.oob_score_)
# Evaluado con aquellos items que no fueron utilizados para crear cada uno de los arboles 1/3 de datos


[0]
% de items clasificados correctamente 1.0
% de items clasificados correctamente 0.9958677685950413
